In [27]:
import tensorflow as tf
import pathlib
import PIL
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

AUTOTUNE = tf.data.AUTOTUNE # prompts the tf.data runtime to tune the value dynamically at runtime

In [2]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#
#
# tf.debugging.set_log_device_placement(True)
#
# # Create some tensors
# a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
# b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
# c = tf.matmul(a, b)
#
# print(c)
# print("cusu")


In [3]:
data_dir = pathlib.Path('Classifier/mnist_png/training')
print(data_dir)
zeros = list(data_dir.glob('0/*.png'))
print(f'{len(zeros)} {str(zeros[0])}')
PIL.Image.open(str(zeros[0]))





Classifier\mnist_png\training
5923 Classifier\mnist_png\training\0\1.png


In [4]:
batch_size = 32
img_height = 28
img_width = 28


In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale'
  )

Found 60000 files belonging to 10 classes.
Using 48000 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')


Found 60000 files belonging to 10 classes.
Using 12000 files for validation.


In [28]:
# Not using image_dataset_from_directory

list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'), shuffle=False)
image_count = len(list_ds)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)
for f in list_ds.take(5):
  print(f.numpy())


class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size) # skips x initial elements from this dataset.
val_ds = list_ds.take(val_size)

print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())




b'Classifier\\mnist_png\\training\\5\\44642.png'
b'Classifier\\mnist_png\\training\\4\\15943.png'
b'Classifier\\mnist_png\\training\\1\\24996.png'
b'Classifier\\mnist_png\\training\\8\\34020.png'
b'Classifier\\mnist_png\\training\\3\\7433.png'
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
48000
12000


In [29]:
# function that converts a file path to an (img, label) pair
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)


def decode_img(img):
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [img_height, img_width])

def process_path(file_path):
  label = get_label(file_path)
  # Load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [30]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)


In [41]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())
    #print(f'->{image}')

    poop = (image.numpy() * 255).astype(np.uint8)

    #data = np.zeros((h, w, 3), dtype=np.uint8)
    #data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
    img = Image.fromarray(poop, 'RGB')
    #img.save('my.png')
    img.show()

  #PIL.Image.open(str(zeros[0]))


Image shape:  (28, 28, 3)
Label:  1


In [ ]:
print(train_ds.class_names)

In [8]:
"""
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(10):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")
"""

'\nplt.figure(figsize=(10, 10))\nfor images, labels in train_ds.take(1):\n  for i in range(10):\n    ax = plt.subplot(4, 4, i + 1)\n    plt.imshow(images[i].numpy().astype("uint8"))\n    plt.title(train_ds.class_names[labels[i]])\n    plt.axis("off")\n'

In [9]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  # print(f'{np.min(image_batch)}, {np.max(image_batch)}')
  # print(labels_batch.shape)

  #print(image_batch[0])

  break



(32, 28, 28, 1)


In [10]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))


0.0 1.0


In [11]:

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [12]:
num_classes = 10

model = tf.keras.Sequential([
  #tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(28, 1, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(28, 1, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(28, 1, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])


In [13]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [14]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)



Epoch 1/3
1500/1500 [==============================] - 195s 125ms/step - loss: 1.3464 - accuracy: 0.5693 - val_loss: 1.0035 - val_accuracy: 0.6562

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 28)        56        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 28)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 28)        812       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 28)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 28)          812       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 28)         0

In [16]:

# Read the image
# img = Image.open('Classifier/mnist_png/testing/0/3.png')
#
# # define a transform to convert the image to grayscale
# transform = transforms.Grayscale()
#
# # apply the above transform on the image
# img = transform(img)
#
# # dispaly the image
# #img.show()
#
# convert_tensor = transforms.ToTensor()
#
# img_tensor = convert_tensor(img)
#
# print(img_tensor.shape)

testing_ds = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path('Classifier/mnist_png/me'),
  seed=123,
  image_size=(img_height, img_width),
  batch_size=1,
  color_mode='grayscale'
  )

testing_normalized_ds = testing_ds.map(lambda x, y: (normalization_layer(x), y))
testing_image_batch, testing_labels_batch = next(iter(testing_normalized_ds))


Found 2 files belonging to 1 classes.


In [17]:
#print(train_ds[0])
model.predict(testing_ds)

2/2 [==============================] - 1s 13ms/step


array([[-0.62816656, -0.6638304 ,  1.0620086 ,  8.020046  ,  0.5115358 ,
         4.7189517 ,  0.2860145 , -2.7044158 ,  0.5547682 , -2.0716217 ],
       [-0.16866863, -2.610769  ,  0.10842181,  2.2507298 , -3.3148282 ,
         2.6609805 ,  1.5462477 , -6.686197  ,  5.628097  , -2.6866364 ]],
      dtype=float32)